### Hyun-Joon Yang
### yanghyun@usc.edu
### QBIO 401
### Assignment 2

#### 1. Write a Python function that takes as input a DNA sequence string and returns its reverse complement string. For example, given an input string “CATGCCGGAATT”, the function returns the reverse complement string “AATTCCGGCATG”

In [1]:
def reverseComplement(seq):
    atgc = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
    complement = ''
    for i in range(len(seq)):
        if seq[len(seq)-i-1] in atgc:
            complement += atgc[seq[len(seq)-i-1]]
        else:
            complement += seq[len(seq)-i-1]
    return complement

#### 2. Write a Python function (or functions) to find all ORFs from six different reading frames (+1, +2, +3, -1, -2, -3) in a sequence string (you may need function #1 to get reverse complement string). Since we are only interested in relatively long ORFs, the function should only return those ORFs longer than an input threshold value. Compute the length of the ORFs as the number of codons (counting the start and stop codon). The start codon is “ATG” and the stop codons are “TAG”, “TAA”, and “TGA”

In the example below, there are two overlapping ORFs (the first three lines are the same sequence and the last three lines are the same reverse complement sequence, with the commas in different places to make it easier to read). The length of the ORF on the first line is seven. The length of the ORF on the second line is six. Each ORF starts with an “ATG”, ends up with a stop
codon, and there is no in-frame stop codon in the sequence.
<br>
<br>
For an input sequence: AATGCCCAAATGCTTTTAAAACCCATGTAGC
<br>
<br>
Frame 1: AAT,GCC,CAA,<b>ATG,CTT,TTA,AAA,CCC,ATG,TAG</b>,C <br>
Frame 2: A,<b>ATG,CCC,AAA,TGC,TTT,TAA</b>,AAC,CCA,TGT,AGC <br>
Frame 3: AA,TGC,CCA,AAT,GCT,TTT,AAA,ACC,CAT,GTA,GC <br>
Frame -1: GCT,ACA,TGG,GTT,TTA,AAA,GCA,TTT,GGG,CAT,T <br>
Frame -2: G,CTA,CAT,GGG,TTT,TAA,AAG,CAT,TTG,GGC,ATT <br>
Frame -3: GC,TAC,ATG,GGT,TTT,AAA,AGC,ATT,TGG,GCA,TT <br>
<br>
The inputs to the function should be the sequence string and the threshold value. The output
should be a list of lists. The length of the list should be the number of ORFs. Each element of the
list should be another list with three numbers describing an individual ORF: the frame number,
the nucleotide position of the first position of the start codon, and the nucleotide position of the
first position of the in-frame stop codon. For the example above, with threshold 5, the output
should be: [[1,9,27],[2,1,16]].

In [2]:
def cleanSeq(seq, index):
    """
    cleans out non codons based on reading frame
    :param seq: DNA sequence
    :param index: start index for reading frame
    :return: new sequence that only contains codons
    """
    # cut out non-codons
    rseq = seq[index:]
    if len(rseq) % 3 != 0:
        rseq = rseq[0:-(len(rseq) % 3)]
    return rseq
    
def readCodon(seq, threshold):
    """
    extracts codons from sequence by reading 3 different frames & finds ORF
    :param seq: DNA sequence
    :param threshold: threshold for ORF length
    :return: list of ORF
    """
    start = 'ATG'
    stop = {'TGA', 'TAA', 'TAG'}
    orfs = []
    # for 3 different frames
    for i in range(3):
        start_index = []
        stop_index = []
        # cut out non-codons
        rseq = cleanSeq(seq, i)
        # look find start & stop codons
        for j in range(len(rseq) // 3):
            codon = rseq[3*j:3*j+3]
            if codon == start:
                start_index.append(3*j+i)
            elif codon in stop:
                stop_index.append(3*j+i)
        # find ORF
        a, b = 0, 0
        while a < len(start_index) and b < len(stop_index):
            if start_index[a] < stop_index[b]:
                if stop_index[b] - start_index[a] >= 3*(threshold-1):
                    orfs.append([start_index[a], stop_index[b]])
                a += 1
            else:
                b += 1
    return orfs

def findORF(seq, threshold):
    orfs = []
    for orf in readCodon(seq, threshold):
        orfs.append([len(orfs)+1, orf[0], orf[1]])
    for orf in readCodon(reverseComplement(seq), threshold):
        orfs.append([len(orfs)+1, orf[0], orf[1]])
    return orfs
            

#### 3. Download the FASTA file “ACE2.fasta” from Blackboard. Use function #2, <i>with threshold 700</i>, to find all ORFs

In [3]:
def openFASTA(filename: str):
    sequence = ''
    with open(filename) as f:
        for line in f:
            # ignore whitespace
            line = line.strip()
            if line == '':
                continue
            # ignore headers & comments
            if line[0] == '>' or line[0] == ';':
                continue
            # ignore end-sequence denotation
            if line[-1] == '*':
                sequence += line[:-1]
            else:
                sequence += line
    return sequence

seq = openFASTA('ACE2.fasta')
findORF(seq, 700)

[[1, 49, 2464], [2, 232, 2464], [3, 292, 2464]]

#### 4. Write a Python function that takes two inputs: a float gc and an integer l. This function returns the expected value E[X] and the probability P(X>L), where X is a geometric random variable representing the number of codons until getting a stop codon

For example, with the inputs (gc=0.4, I=50):
<br> <br>
First, the function calculates p, the probability of getting a stop codon:
<br>
$ 𝑝 = 𝑃(𝑇𝐴𝐺 𝑜𝑟 𝑇𝐴𝐴 𝑜𝑟 𝑇𝐺𝐴) = (0.3 × 0.3 × 0.2) + (0.3 × 0.3 × 0.3) + (0.3 × 0.2 × 0.3)
= 0.063 $
<br> <br>
Second, the function calculates $ 𝐸[𝑋] = 1⁄𝑝 = 15.873 $ <br> <br>
Third, the function calculates $ 𝑃(𝑋 > 𝐿) = 1 − 𝑃(𝑋 ≤ 50) = 1 − 0.961 = 0.039 $ <br> <br>
Finally, the function returns 15.873 and 0.039

In [4]:
from scipy import stats

def calcP(gc, l):
    p = ((1-gc)/2)**3 + 2*(((1-gc)/2)**2)*(gc/2)
    X = stats.geom(p)
    return X.expect(), 1 - X.cdf(l)

#### 5. If you try to optimize your Python code, describe how you improve the performance

Instead of using 2 for loops (one for start codons and one for stop codons), I used a while loop with 2 counters. This way, the algorithm does not spend time on checking stop codons that have already been 'checked'. Therefore, while the for loop algorithm will have a runtime of O(nm), the while loop algorithm will have a runtime of O(n + m) where n is the number of start codons and m is the number of stop codons. 

#### TESTS

In [5]:
import unittest
import numpy as np

class TestReverseComplement(unittest.TestCase):
    def test_default(self):
        seq = "CATGCCGGAATT"
        
        output = reverseComplement(seq)
        
        self.assertEqual('AATTCCGGCATG', output)
        
class TestFindORF(unittest.TestCase):
    def test_default(self):
        seq = 'AATGCCCAAATGCTTTTAAAACCCATGTAGC'
        
        orfs = findORF(seq, 5)
        
        self.assertEqual([[1,9,27], [2,1,16]], orfs)
    
    def test_overlap(self):
        seq = "ATG...ATG...TAG"
        
        orfs = findORF(seq, 1)
        
        self.assertEqual([[1,0,12],[2,6,12]], orfs)
        
    def test_overlap2(self):
        seq = "ATG...ATG...TAG...TAG"
        
        orfs = findORF(seq, 1)
        
        self.assertEqual([[1,0,12],[2,6,12]], orfs)
        
    def test_noEnd(self):
        seq = 'ATG...'
        
        orfs = findORF(seq, 1)
        
        self.assertEqual([], orfs)
    
    def test_multipleEnds(self):
        seq = 'ATG...TAG...TAG'
        
        orfs = findORF(seq, 1)
        
        self.assertEqual([[1,0,6]], orfs)
        
    def test_noStart(self):
        seq = '...TAG'
        
        orfs = findORF(seq, 1)
        
        self.assertEqual([], orfs)
        
    def test_multipleORF(self):
        seq = 'ATG...TAG...ATG...TAG...ATG...TAG'
        
        orfs = findORF(seq, 1)
        
        self.assertEqual([[1,0,6],[2,12,18],[3,24,30]], orfs)
    
    def test_threshold(self):
        threshold = 3
        seq = 'ATGTAG'
        
        orfs = findORF(seq, threshold)
        
        self.assertEqual([], orfs)
        
class TestCalcP(unittest.TestCase):
    def test_default(self):
        a, b = calcP(0.4, 50)
        
        np.testing.assert_almost_equal(a, 15.8730158)
        np.testing.assert_almost_equal(b, 0.038634878)    
    
unittest.main(argv=[''], verbosity=2, exit=False)

test_default (__main__.TestCalcP) ... ok
test_default (__main__.TestFindORF) ... ok
test_multipleEnds (__main__.TestFindORF) ... ok
test_multipleORF (__main__.TestFindORF) ... ok
test_noEnd (__main__.TestFindORF) ... ok
test_noStart (__main__.TestFindORF) ... ok
test_overlap (__main__.TestFindORF) ... ok
test_overlap2 (__main__.TestFindORF) ... ok
test_threshold (__main__.TestFindORF) ... ok
test_default (__main__.TestReverseComplement) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.032s

OK
